In [1]:
import numpy as np
import networkx as nx
from SNN_neat import DAG, plot_dag
from SNN_neat import ReLU, LeakyReLU, Sigmoid, Linear, Tanh

def dummy_uuid_generator():
    i = 0
    while True:
        yield str(i)
        i += 1

dummy_gen = dummy_uuid_generator()

def uuid1():
    return next(dummy_gen)

In [2]:
INPUTS = 2
OUTPUTS = 1

# test creation of DAG
dag = DAG(INPUTS, OUTPUTS)
plot_dag(dag)

In [3]:
# test adding connections
for n1_id in dag.input_nodes:
    for n2_id in dag.output_nodes:
        n1 = dag.get_node(n1_id)
        n2 = dag.get_node(n2_id)
        dag.add_connection(n1.id, n2.id)
plot_dag(dag)

In [4]:
# test getting conections and connection check
connections = dag.get_connections()
print("Created Connections", connections)
print("Check if connection can be found:", dag.is_connected(dag.input_nodes[0], dag.output_nodes[0]))

Created Connections [('0', '2'), ('1', '2')]
Check if connection can be found: True


In [5]:
# test adding nodes
for connection in connections:
    print(f"Adding Node between {(connection[0], connection[1])}", dag.add_node(connection[0], connection[1]))
plot_dag(dag)

Adding Node between ('0', '2') (True, '3')
Adding Node between ('1', '2') (True, '4')


In [6]:
dag.add_connection('3', '4')
 
plot_dag(dag)

In [7]:
dag.add_node('3', '4')
 
plot_dag(dag)

In [8]:
dag.add_node('0', '3')
 
plot_dag(dag)

In [9]:
 # test getting processing order
print("Processing Order:", dag.get_processing_order())
 
plot_dag(dag)

Processing Order: [<SNN_neat.Layer object at 0x0000028CE5FF6AD0>, <SNN_neat.Layer object at 0x0000028CE5FF6C10>, <SNN_neat.Layer object at 0x0000028CE5FF6B50>, <SNN_neat.Layer object at 0x0000028CE5FF6D10>, <SNN_neat.Layer object at 0x0000028CE5FF69D0>, <SNN_neat.Layer object at 0x0000028CE5FF6790>]


In [10]:
print("Starting Cons:", dag.get_connections())

dag.remove_node('3')
    

print("Ending Cons:", dag.get_connections())
 
plot_dag(dag)

Starting Cons: [('0', '6'), ('1', '4'), ('3', '2'), ('3', '5'), ('4', '2'), ('5', '4'), ('6', '3')]
Ending Cons: [('0', '6'), ('1', '4'), ('4', '2'), ('5', '4')]


In [11]:
dag.kill_floaters()
plot_dag(dag)

In [12]:
dag.add_connection('0', '4')
ok, new_id = dag.add_node('0', '4')
dag.add_connection('4', new_id)
dag.add_node('4','2')
plot_dag(dag)

In [13]:
dag.remove_connection('4', '7')
plot_dag(dag)

# Mutation Test

In [14]:
for i in range(10):
    dag.mutate()   
    plot_dag(dag)

In [15]:
print(dag.get_legal_connections())

[('0', '2'), ('0', '4'), ('0', '8'), ('0', '9'), ('0', '10'), ('0', '12'), ('0', '13'), ('0', '14'), ('0', '15'), ('1', '2'), ('1', '8'), ('1', '9'), ('1', '10'), ('1', '11'), ('1', '12'), ('1', '13'), ('1', '14'), ('1', '15'), ('4', '2'), ('4', '7'), ('7', '2'), ('7', '8'), ('7', '13'), ('7', '14'), ('8', '4'), ('8', '14'), ('9', '2'), ('9', '4'), ('9', '7'), ('9', '8'), ('9', '12'), ('9', '13'), ('9', '14'), ('9', '15'), ('10', '2'), ('10', '4'), ('10', '7'), ('10', '8'), ('10', '9'), ('10', '13'), ('10', '14'), ('11', '2'), ('11', '4'), ('11', '7'), ('11', '8'), ('11', '9'), ('11', '10'), ('11', '12'), ('11', '13'), ('11', '14'), ('12', '2'), ('12', '4'), ('12', '7'), ('12', '8'), ('12', '10'), ('12', '14'), ('13', '2'), ('13', '4'), ('13', '8'), ('13', '12'), ('13', '14'), ('14', '4'), ('14', '8'), ('15', '2'), ('15', '4'), ('15', '7'), ('15', '8'), ('15', '10'), ('15', '11'), ('15', '12'), ('15', '13'), ('15', '14')]


# test space

In [16]:
from keras.datasets import mnist, boston_housing
import matplotlib.pyplot as plt
# Load the MNIST dataset
""" (x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28*28) / 255
x_test = x_test.reshape(-1, 28*28) / 255

# convert Y to one-hot encoding
y_train = np.eye(10)[y_train] """

# Load the Boston Housing dataset
(x_train, y_train), (x_test, y_test) = boston_housing.load_data()
#x_train = x_train 
y_train = y_train.reshape(-1, 1)/ np.max(y_train) 

In [17]:
mnist_dag = DAG(x_train.shape[1], y_train.shape[1], fully_connect=True)

""" # add 10 hidden nodes
for i in range(10):
    # add connection from input to output
    mnist_dag.add_connection(mnist_dag.input_nodes[0], mnist_dag.output_nodes[0])
    ok, new_node_id = mnist_dag.add_node(mnist_dag.input_nodes[0], mnist_dag.output_nodes[0])

    print("New Node ID:", new_node_id)

    # connect to all remaining input and output nodes
    for inp in mnist_dag.input_nodes[1:]:
        mnist_dag.add_connection(inp, new_node_id)
    for out in mnist_dag.output_nodes[1:]:
        mnist_dag.add_connection(new_node_id, out) """

plot_dag(mnist_dag)

In [18]:
losses = mnist_dag.train(x_train, y_train, epochs=3, lr=0.01, verbose=True)

Start Training DAG 0 with 3 epochs


  0%|          | 0/3 [00:00<?, ?it/s]


AttributeError: 'Node' object has no attribute 'out_val'

In [ ]:
# plot losses
plt.plot(losses)
plt.show()

In [ ]:
yHat = mnist_dag.process(x_train)
for i in range(10):
    print(np.around(yHat[i], 2), y_train[i],  y_train[i] - yHat[i])

In [ ]:
for i in range(10):
    #plt.imshow(x_test[i].reshape(28, 28))
    #plt.show()
    
    output = mnist_dag.process([x_train[i]])
    print("YHat:", output, "Y:", y_train[i])
    #print(np.argmax(output))

In [ ]:
def custom_node_backprop(node, errors, input_values, lr) -> list:
    # internal error
    z_prime = node.activation_function.derivative(node.int_val)
    e_int = errors * z_prime
    print("E_int:", e_int.shape, "Errors:", errors.shape, "Activation Derivative:", z_prime.shape, "Int Val:", node.int_val.shape)

    # update bias
    if node.has_bias:
        node.bias -= lr * np.sum(e_int)
    # update weights and return errors for input nodes
    prop_e = []
    for i, n_id in enumerate(node.input_cons):
        prop_e.append((n_id, np.dot(e_int, node.weights[i].T)))

        print("E_int:", e_int.shape, "Input:", input_values[i].shape, "Change in Weights:", np.sum(lr * np.dot(input_values[i], e_int)))
        node.weights[i] -= np.sum(lr * np.dot(input_values[i], e_int))

    return prop_e

def custom_backprop(dag, outputs, targets, lr): 
    #print("Targets:", list(targets), "Outputs", list(outputs))
    error_dict = {}
    for node_id in dag.output_nodes:
        node = dag.get_node(node_id)
        node_idx = dag.output_nodes.index(node_id)
        print("Node index for output node:", node_idx)
        note_targets = targets[:, node_idx]
        error = dag.error_function.derivative(node.out_val, note_targets)
        print("Error on node", node_id, error.shape)
        error_dict[node_id] = error

    for layer in reversed(dag.processing_order):
        for node_id in layer:
            # skip input nodes
            if node_id in dag.input_nodes:
                continue
            
            node = dag.get_node(node_id)
            
            # get the error for the node
            error = error_dict[node_id]

            # get the input values of the node
            input_vals = np.array([dag.get_node(n_id).out_val for n_id in node.input_cons])

            # backpropagate the error
            prop_err = custom_node_backprop(node, error, input_vals, lr)
            # add the propagated error to the error dict
            for n_id, e in prop_err:
                if n_id in error_dict:
                    error_dict[n_id] += e
                else:
                    error_dict[n_id] = e

In [ ]:
output = mnist_dag.process(x_train)

loss = mnist_dag.cost(output, y_train)
print("Loss:", loss)

# do manual step by step backprop to see by what values the weights should be updated
cost = custom_backprop(mnist_dag, output, y_train, 0.01)



